In [4]:
import base64
import json
import boto3
from pathlib import Path

# -------------------------
# Config
# -------------------------
REGION = "eu-north-1"
ENDPOINT_NAME = "feature-extraction-efficientnetb7-6"
IMAGE_PATH = "front.png"

runtime = boto3.client("sagemaker-runtime", region_name=REGION)

# -------------------------
# Helper
# -------------------------
def to_b64(path: str) -> str:
    b64 = base64.b64encode(Path(path).read_bytes()).decode("utf-8")
    return "".join(b64.split())

# -------------------------
# Invoke
# -------------------------
b64 = to_b64(IMAGE_PATH)

payload = {"instances": [{"image_bytes": {"b64": b64}}]}
body = json.dumps(payload).encode("utf-8")

resp = runtime.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Accept="application/json",
    Body=body,
)

raw = resp["Body"].read().decode("utf-8")
print("Raw response (first 300 chars):")
print(raw[:300])

# -------------------------
# Parse
# -------------------------
data = json.loads(raw)
p0 = data["predictions"][0]

# If predictions[0] is dict -> {"features":[...]}
# If predictions[0] is list -> [...]
feats = p0["features"] if isinstance(p0, dict) and "features" in p0 else p0

print("\nFeature length:", len(feats))
print("First 10 features:", feats[:10])


Raw response (first 300 chars):
{    "predictions": [[0.0693255588, -0.0671755522, -0.15216817, 0.334834099, 0.220378801, -0.095134221, 0.00738465693, -0.0282613188, 0.0919002593, 0.183095, -0.055449076, -0.129393876, -0.131789297, -0.0919747, 0.0377806, 0.0185107477, -0.161865503, -0.118041612, -0.133932009, -0.0872334614, -0.166

Feature length: 2560
First 10 features: [0.0693255588, -0.0671755522, -0.15216817, 0.334834099, 0.220378801, -0.095134221, 0.00738465693, -0.0282613188, 0.0919002593, 0.183095]
